### 187_ferry_stops


* [issue 187](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/187)
* [this notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/tree/main/notebook/187_ferry_stops.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-11-05 05:43


In [2]:
#!/usr/bin/env python3
"""
Generisk OSM↔Wikidata-koll via valfri SPARQL

Syfte
------
Skicka in *vilken SPARQL-fråga som helst* som returnerar ett ?item (Wikidata-objekt)
och **någon** form av OSM-id (node/way/relation) – t.ex. via standardegenskaperna
P10689 (node), P11693 (way), P402 (relation) eller som färdiga URL:er – så kontrollerar
skriptet att **motsvarande OSM-objekt pekar tillbaka** till Wikidata via `wikidata=Q…`.

Vad gör skriptet?
-----------------
1) Kör din SPARQL, tolkar resultatet och extraherar:
   - QID (från ?item)
   - OSM-ID:n (node/way/relation) från valfria fält/format.
2) Hämtar nämnda OSM-objekt direkt via Overpass (per id, ej bbox).
3) Validerar back-link:
   - ✅ **OK**: OSM har `wikidata`-tagg som *innehåller* samma QID (hanterar även semikolon-separerade listor).
   - ⚠️ **SAKNAS**: OSM saknar `wikidata`-tag helt.
   - ❌ **FEL**: OSM har `wikidata`, men den matchar **inte** QID.
4) Skriver ut **listor med klickbara länkar** för varje kategori och exporterar CSV.

Exempelanrop
------------
- `run_generic(SPARQL_QUERY)`

Notera
------
- Du kan fortfarande använda den tidigare `run()` för den hårdkodade SAT-frågan om du vill.
"""
from __future__ import annotations
import re
import math
import json
from typing import Dict, Iterable, List, Tuple

import pandas as pd
import requests

WIKIDATA_SPARQL = "https://query.wikidata.org/sparql"
OVERPASS_API = "https://overpass-api.de/api/interpreter"
USER_AGENT = "SAT-OSM-WD-Consistency/2.0 (contact: your-email@example.com)"

# -------------------------
# Hjälpfunktioner
# -------------------------

def make_headers() -> Dict[str, str]:
    return {"User-Agent": USER_AGENT}


def to_osm_prefixed_id(typ: str, osm_id: int) -> str:
    return {"node": "n", "way": "w", "relation": "r"}[typ] + str(int(osm_id))


def parse_osm_from_value(val: str) -> Tuple[str, int] | None:
    """Försök tolka ett OSM-id (typ, id) från godtycklig sträng/URL/tal.
    Stödjer t.ex. 12345, "12345", "https://www.openstreetmap.org/node/12345".
    Returnerar (typ, id) eller None.
    """
    if val is None:
        return None
    s = str(val).strip()

    # URL-format
    m = re.search(r"openstreetmap\.org\/(node|way|relation)\/(\d+)", s)
    if m:
        return m.group(1), int(m.group(2))

    # Rena tal: lämna typ okänd
    if s.isdigit():
        # Typ avgörs av fältnamnet – hanteras av anroparen
        return ("unknown", int(s))

    return None


# -------------------------
# Wikidata
# -------------------------

def fetch_wikidata_generic(sparql: str) -> pd.DataFrame:
    """Kör godtycklig SPARQL och extraherar QID och OSM-id:n ur valfria kolumner.
    Krav: ?item måste finnas och vara ett WD-objekt.
    Heuristik för OSM:
      - Leta kolumnnamn som innehåller något av: node|way|rel|osm|OSM
      - För varje sådan kolumn: tolka värdet som URL eller siffra.
      - Om typ är "unknown" – gissa typ utifrån kolumnnamn (node/way/relation).
    Returnerar DF med kolumner: qid, label (om finns), osm_type, osm_id
    (en rad per QID×OSM-referens). Om ingen OSM-referens hittas för en QID
    returneras ändå en rad utan OSM-typ/id för spårbarhet.
    """
    r = requests.get(WIKIDATA_SPARQL, params={"query": sparql, "format": "json"}, headers=make_headers())
    r.raise_for_status()
    rows = r.json()["results"]["bindings"]

    out_rows: List[Dict] = []
    for b in rows:
        item_uri = b.get("item", {}).get("value")
        if not item_uri:
            continue
        qid = item_uri.rsplit("/", 1)[-1]
        label = b.get("itemLabel", {}).get("value")

        # samla alla kandidater till OSM-kolumner
        osm_keys = [k for k in b.keys() if re.search(r"(node|way|rel|osm)", k, re.I) and k != "item"]

        found_any = False
        for k in osm_keys:
            raw = b[k].get("value")
            parsed = parse_osm_from_value(raw)
            if parsed:
                typ, oid = parsed
                if typ == "unknown":
                    # gissa från kolumnnamn
                    lk = k.lower()
                    if "node" in lk:
                        typ = "node"
                    elif "way" in lk:
                        typ = "way"
                    elif "rel" in lk:
                        typ = "relation"
                    else:
                        # kan ej avgöra – hoppa
                        continue
                out_rows.append({
                    "qid": qid, "label": label, "osm_type": typ, "osm_id": int(oid),
                    "osm_url": f"https://www.openstreetmap.org/{typ}/{int(oid)}",
                })
                found_any = True

        if not found_any:
            # lägg en rad utan OSM för spårbarhet
            out_rows.append({"qid": qid, "label": label, "osm_type": None, "osm_id": None, "osm_url": None})

    df = pd.DataFrame(out_rows).drop_duplicates()
    return df


# -------------------------
# Overpass
# -------------------------

def fetch_osm_by_ids(df_refs: pd.DataFrame) -> pd.DataFrame:
    """Hämta OSM-objekt per id för de rader som har osm_type/osm_id.
    Returnerar kolumner: osm_type, osm_id, url, wikidata_tag
    """
    subset = df_refs.dropna(subset=["osm_type", "osm_id"]).copy()
    if subset.empty:
        return pd.DataFrame(columns=["osm_type","osm_id","url","wikidata_tag"])   

    ids_by_type: Dict[str, List[int]] = {"node": [], "way": [], "relation": []}
    for _, r in subset.iterrows():
        t = r["osm_type"]
        if t in ids_by_type:
            ids_by_type[t].append(int(r["osm_id"]))

    def batched(lst: List[int], n: int = 200) -> Iterable[List[int]]:
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    rows: List[Dict] = []
    for t in ("node","way","relation"):
        if not ids_by_type[t]:
            continue
        for batch in batched(sorted(set(ids_by_type[t]))):
            ids_str = ",".join(str(i) for i in batch)
            q = f"""
            [out:json][timeout:50];
            {t}(id:{ids_str});
            out ids tags center;
            """.strip()
            rr = requests.post(OVERPASS_API, data={"data": q}, headers=make_headers())
            rr.raise_for_status()
            for el in rr.json().get("elements", []):
                tags = el.get("tags", {})
                rows.append({
                    "osm_type": t,
                    "osm_id": el.get("id"),
                    "url": f"https://www.openstreetmap.org/{t}/{el.get('id')}",
                    "wikidata_tag": tags.get("wikidata"),
                    "name": tags.get("name"),
                })

    return pd.DataFrame(rows)


# -------------------------
# Validering & utskrift
# -------------------------

def normalize_wikidata_values(val: str | None) -> List[str]:
    if not val:
        return []
    # Dela på semikolon/komma och trimma, ta bara sådant som ser ut som Q…
    parts = re.split(r"[;|,]", val)
    return [p.strip() for p in parts if re.match(r"^Q\d+$", p.strip(), re.I)]


def verify_backlinks(df_refs: pd.DataFrame, df_osm: pd.DataFrame) -> Dict[str, pd.DataFrame]:
    # join på (osm_type, osm_id)
    key = ["osm_type","osm_id"]
    merged = pd.merge(df_refs.dropna(subset=key), df_osm, on=key, how="left", suffixes=("","_osm"))

    ok_rows = []
    missing_rows = []
    wrong_rows = []

    for _, r in merged.iterrows():
        qid = r["qid"]
        tag = r.get("wikidata_tag")
        if pd.isna(tag) or not tag:
            missing_rows.append(r)
        else:
            vals = normalize_wikidata_values(tag)
            if qid in vals:
                ok_rows.append(r)
            else:
                wrong_rows.append(r)

    return {
        "ok": pd.DataFrame(ok_rows),
        "missing": pd.DataFrame(missing_rows),
        "wrong": pd.DataFrame(wrong_rows),
        "all_refs": df_refs,
        "all_osm": df_osm,
    }


def print_link_lists(res: Dict[str, pd.DataFrame]):
    def print_group(title: str, df: pd.DataFrame):
        print(f"\n{title}")
        print("-" * len(title))
        if df.empty:
            print("(inget)")
            return
        # Grupp: per QID
        for qid, grp in df.groupby("qid"):
            print(f"{qid}:")
            for _, r in grp.iterrows():
                print(f"  {r['url']}")

    print_group("❌ OSM har wikidata men pekar fel", res["wrong"])        
    print_group("✅ OSM back-link OK (wikidata matchar QID)", res["ok"])    
    print_group("⚠️ OSM saknar wikidata-tag (lägg till QID)", res["missing"]) 


# -------------------------
# Publika körfunktioner
# -------------------------

def run_generic(sparql_query: str, export_prefix: str = "wd_osm_backlinks"):
    print("Kör SPARQL…")
    refs = fetch_wikidata_generic(sparql_query)
    print(f"Rader från SPARQL (QID×OSM-ref): {len(refs)}")

    print("Hämtar OSM per id…")
    osm = fetch_osm_by_ids(refs)
    print(f"OSM-objekt hämtade: {len(osm)}")

    res = verify_backlinks(refs, osm)

    # Exportera CSV
    for name, df in res.items():
        path = f"{export_prefix}_{name}.csv"
        df.to_csv(path, index=False)
        print(f"↳ sparade {path} ({len(df)} rader)")

    # Listrapport med klickbara länkar
    print_link_lists(res)



def run_sat_example():
    run_generic(SAT_SPARQL, export_prefix="sat_backlinks")


In [3]:
# https://w.wiki/FSNg
run_generic("""
SELECT ?etapp ?etappLabel ?item ?itemLabel ?coord ?googlemap ?img ?OSMnode ?OSMway ?OSMrel WHERE {
  ?etapp wdt:P361 wd:Q131318799.       # Etapper del av SAT
  OPTIONAL {
    ?etapp wdt:P1427 ?item.           # Startpunkt
    OPTIONAL { ?item wdt:P625 ?coord. } # Koordinat för startpunkten
    OPTIONAL  { ?item wdt:P18 ?img. } # Bild
    OPTIONAL { ?item wdt:P3749 ?gid.} # Google Map id 
    OPTIONAL { ?item wdt:P10689 ?OSMway }
    OPTIONAL { ?item wdt:P11693 ?OSMnode }
    OPTIONAL { ?item wdt:P402   ?OSMrel }
    BIND(URI(CONCAT("https://www.google.com/maps?cid=",?gid)) AS ?googlemap)
   }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?etapp
""", export_prefix="sat_backlinks")


Kör SPARQL…
Rader från SPARQL (QID×OSM-ref): 41
Hämtar OSM per id…
OSM-objekt hämtade: 41
↳ sparade sat_backlinks_ok.csv (39 rader)
↳ sparade sat_backlinks_missing.csv (1 rader)
↳ sparade sat_backlinks_wrong.csv (1 rader)
↳ sparade sat_backlinks_all_refs.csv (41 rader)
↳ sparade sat_backlinks_all_osm.csv (41 rader)

❌ OSM har wikidata men pekar fel
--------------------------------
Q134369914:
  https://www.openstreetmap.org/node/5530966

✅ OSM back-link OK (wikidata matchar QID)
-----------------------------------------
Q133867299:
  https://www.openstreetmap.org/node/1823005981
Q133884655:
  https://www.openstreetmap.org/way/39504808
Q133884690:
  https://www.openstreetmap.org/node/7683934978
Q133884785:
  https://www.openstreetmap.org/node/424056139
Q133884836:
  https://www.openstreetmap.org/node/269395949
Q133885585:
  https://www.openstreetmap.org/node/1611299356
Q133886292:
  https://www.openstreetmap.org/node/435187426
Q133993978:
  https://www.openstreetmap.org/node/435221450
Q

In [4]:
# =========================
# KONFIG
# =========================
SAT_GEOJSON_PATH   = "SAT_full.geojson"
OUTPUT_PREFIX      = "187_WD_OSM_ferry_stops"
FETCH_FROM_WD      = True

# SPARQL: ferry stops with coordinates, OSM links, images https://w.wiki/FT4Y
SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?OSMnode ?OSMway ?OSMrel 
       ?etapp ?etappLabel ?googlemap ?type ?typeLabel ?kartIcon ?hex
WHERE {
  ?etapp wdt:P361 wd:Q131318799.
  ?etapp wdt:P1427 ?item.
  ?item wdt:P31 ?type.

  OPTIONAL { ?item wdt:P625 ?coord. }
  OPTIONAL { ?item wdt:P18 ?img. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P402   ?OSMrel }
  OPTIONAL { ?item wdt:P3749 ?gid. 
  BIND(URI(CONCAT("https://www.google.com/maps?cid=",?gid)) AS ?googlemap) }

  # fetch Kartographer info from type
  OPTIONAL { ?type wdt:P1343 ?kartIcon. }
  OPTIONAL { ?type wdt:P465 ?hex. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?etapp ?itemLabel
"""


In [5]:
def fetch_wd_to_csv(sparql: str, out_csv: str):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.2 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": sparql}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "OSMnode": gv("OSMnode"),
            "OSMway": gv("OSMway"),
            "OSMrel": gv("OSMrel"),
            "etapp": gv("etapp"),
            "etappLabel": gv("etappLabel"),
            "googlemap": gv("googlemap"),
        })

    df = pd.DataFrame(rows)

    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"] = lats
    df["lon"] = lons
    df.to_csv(out_csv, index=False)
    return df


In [6]:
if FETCH_FROM_WD:
    print("Hämtar färjelägen från Wikidata…")
    df_ferry = fetch_wd_to_csv(SPARQL, "ferry_stops_wd.csv")
else:
    if not os.path.exists("ferry_stops_wd.csv"):
        raise FileNotFoundError("Hittar inte ferry_stops_wd.csv – kör FETCH_FROM_WD=True eller lägg filen bredvid scriptet.")
    df_ferry = pd.read_csv("ferry_stops_wd.csv")

print(f"Färjelägen: {len(df_ferry)}")


Hämtar färjelägen från Wikidata…
Färjelägen: 43


In [7]:
import os, json, html
import requests
import pandas as pd
import folium
from folium.plugins import Fullscreen, MiniMap, MeasureControl, MousePosition
import html as _html
import html
from string import Template
from datetime import datetime



# =========================
# KONFIG
# =========================
SAT_GEOJSON_PATH   = "SAT_full.geojson"
OUTPUT_PREFIX      = "187_WD_OSM_ferry_stops"
FETCH_FROM_WD      = True

SPARQL = r"""
SELECT ?item ?itemLabel ?coord ?img ?OSMnode ?OSMway ?OSMrel 
       ?etapp ?etappLabel ?googlemap ?type ?typeLabel ?kartIcon ?hex
WHERE {
  ?etapp wdt:P361 wd:Q131318799.
  ?etapp wdt:P1427 ?item.
  ?item wdt:P31 ?type.

  OPTIONAL { ?item wdt:P625 ?coord. }
  OPTIONAL { ?item wdt:P18 ?img. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P402   ?OSMrel }
  OPTIONAL { ?item wdt:P3749 ?gid. 
             BIND(URI(CONCAT("https://www.google.com/maps?cid=",?gid)) AS ?googlemap) }

  OPTIONAL { ?type wdt:P1343 ?kartIcon. }
  OPTIONAL { ?type wdt:P465 ?hex. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?etapp ?itemLabel
"""

# Kartographer → FontAwesome mapping
KARTO_MAP = {
    "ferry": "ship",
    "toilets": "restroom",
    "drinking-water": "tint",
    "shelter": "home",
    "fire": "fire",
    "information": "info-sign"
}

# =========================
# HELPERS
# =========================
def safe(s):
    return None if (s is None or (isinstance(s, float) and pd.isna(s))) else str(s)

def fetch_wd_to_csv(sparql: str, out_csv: str):
    url = "https://query.wikidata.org/sparql"
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "SAT-map-builder/1.3 (contact: salgo60@msn.com)"
    }
    r = requests.get(url, params={"query": sparql}, headers=headers, timeout=80)
    r.raise_for_status()
    data = r.json()

    rows = []
    for b in data["results"]["bindings"]:
        gv = lambda k: b[k]["value"] if k in b else None
        rows.append({
            "item": gv("item"),
            "itemLabel": gv("itemLabel"),
            "coord": gv("coord"),
            "img": gv("img"),
            "OSMnode": gv("OSMnode"),
            "OSMway": gv("OSMway"),
            "OSMrel": gv("OSMrel"),
            "etapp": gv("etapp"),
            "etappLabel": gv("etappLabel"),
            "googlemap": gv("googlemap"),
            "type": gv("type"),
            "typeLabel": gv("typeLabel"),
            "kartIcon": gv("kartIcon"),
            "hex": gv("hex"),
        })

    df = pd.DataFrame(rows)

    # Ensure coord column exists
    if "coord" not in df.columns:
        df["coord"] = None
        print("⚠️ No coord values returned from SPARQL")
        print("Available columns:", df.columns.tolist())
        print(df.head())
    
    # parse coord "Point(lon lat)"
    lats, lons = [], []
    for v in df["coord"].astype(str):
        lat = lon = None
        if v.startswith("Point(") and v.endswith(")"):
            parts = v[6:-1].split()
            if len(parts) == 2:
                try:
                    lon = float(parts[0]); lat = float(parts[1])
                except:
                    pass
        lats.append(lat); lons.append(lon)
    df["lat"] = lats
    df["lon"] = lons
    df.to_csv(out_csv, index=False)
    return df

def bounds_from_geojson(gj):
    minlat=minlon= 1e9
    maxlat=maxlon=-1e9
    def add(coords):
        nonlocal minlat, minlon, maxlat, maxlon
        if isinstance(coords[0], (float,int)):
            lon, lat = coords
            minlat = min(minlat, lat); maxlat = max(maxlat, lat)
            minlon = min(minlon, lon); maxlon = max(maxlon, lon)
        else:
            for c in coords:
                add(c)
    for feat in gj.get("features", []):
        geom = feat.get("geometry", {})
        add(geom.get("coordinates", []))
    if minlat == 1e9:
        return (59.3, 18.0, 59.6, 19.2)
    return (minlat, minlon, maxlat, maxlon)
def add_about_box(
    m,
    issue_number: int,
    map_name: str,
    created_date: str | None = None,
    repo: str = "salgo60/Stockholm_Archipelago_Trail",
    collapsed: bool = False,
):
    """
    Notebook/Edge-säker About-box med string.Template ($-placeholder).
    """
    if created_date is None:
        created_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    map_dom_id = m.get_name()
    box_id     = f"sat-about-{map_dom_id}"
    header_id  = f"{box_id}-hdr"
    issue_url  = f"https://github.com/{repo}/issues/{issue_number}"

    links = [
        ("▶️ Video about the ferry stop map", "https://youtu.be/k55c4ru9y5c"),
        ("SAT Dashboard", "dashboard.html"),
        ("Project repo issues", "https://github.com/salgo60/Stockholm_Archipelago_Trail/issues?q=is%3Aissue"),
        ("Trail on OSM (rel 19012437)", "https://www.openstreetmap.org/relation/19012437"),
        ("Trail on Wikicommons", "https://commons.wikimedia.org/wiki/Category:Stockholm_Archipelago_Trail"),
        ("Official page", "https://stockholmarchipelagotrail.com/"),
        ("Unofficial FB group", "https://www.facebook.com/groups/2875020699552247"),
        ("Visit Sweden", "https://traveltrade.visitsweden.com/plan/news-sweden/Stockholm-Archipelago-Trail/"),

    ]
    links_html = "".join(
        f'<div><a href="{_html.escape(u)}" target="_blank" style="text-decoration:none;">🔗 {_html.escape(t)}</a></div>'
        for t, u in links
    )
    collapsed_class = "sat-about-collapsed" if collapsed else ""

    tpl = Template(r"""
<style>
  .sat-about { position: fixed; z-index: 10000; background: rgba(255,255,255,0.97);
    border: 2px solid #666; border-radius: 10px; box-shadow: 0 2px 6px rgba(0,0,0,0.25);
    font: 12px/1.35 -apple-system, system-ui, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    pointer-events: auto; min-width: 240px; max-width: 320px; }
  .sat-about-header { cursor: pointer; padding: 8px 10px; font-weight: 700;
    display: flex; align-items: center; gap: 6px; user-select: none;
    background: rgba(248,248,248,.9); border-bottom: 1px solid #e5e7eb; }
  .sat-about-body { padding: 8px 10px 10px 10px; }
  .sat-about-collapsed .sat-about-body { display: none; }
  .sat-chevron { margin-left: auto; transition: transform .15s ease-in-out; }
  .sat-about-collapsed .sat-chevron { transform: rotate(-90deg); }
  .sat-links { margin-top: 6px; padding-top: 6px; border-top: 1px solid #e5e7eb; }
  @media (max-width: 640px) { .sat-about { font-size: 11px; min-width: 200px; max-width: 260px; } }
</style>

<div id="$box_id" class="sat-about $collapsed_class">
  <div id="$header_id" class="sat-about-header" title="Click to collapse/expand">
    <span>ℹ️ About</span>
    <span class="sat-chevron">▸</span>
  </div>
  <div class="sat-about-body">
    <div style="font-weight:700;margin-bottom:4px;">Stockholm Archipelago Trail Map</div>
    <div>Issue: <a href="$issue_url" target="_blank">#$issue_number</a>&nbsp;&nbsp; Map: $map_name</div>
    <div>Created: $created_date</div>
    <div>Latest updates: saved as <i>_latest.html</i></div>
    <div class="sat-links">$links_html</div>
  </div>
</div>

<script>
(function(){
  var mapId = "$map_dom_id";
  var boxId = "$box_id";
  var hdrId = "$header_id";
  var storageKey = "satAboutCollapsed_" + "$map_dom_id" + "_#$issue_number";

  function setCollapsed(box, collapsed) {
    var body = box.querySelector(".sat-about-body");
    if (collapsed) { box.classList.add("sat-about-collapsed"); if (body) body.style.display = "none"; }
    else { box.classList.remove("sat-about-collapsed"); if (body) body.style.display = "block"; }
    try { localStorage.setItem(storageKey, collapsed ? "1" : "0"); } catch(e) {}
  }

  function placeBox(mapEl, box) {
    var zoom = mapEl.querySelector(".leaflet-control-zoom");
    var mr   = mapEl.getBoundingClientRect();
    var top = 10, left = 10;
    if (zoom) {
      var zr = zoom.getBoundingClientRect();
      top  = (zr.bottom - mr.top) + 8;
      left = (zr.left   - mr.left) + zr.width + 8;
    }
    box.style.top  = top  + "px";
    box.style.left = left + "px";
  }

  function init(tries) {
    tries = tries || 0;
    var mapEl = document.getElementById(mapId);
    var box   = document.getElementById(boxId);
    var hdr   = document.getElementById(hdrId);
    if (!mapEl || !box || !hdr) { if (tries < 60) return setTimeout(function(){ init(tries+1); }, 120); return; }

    try {
      var stored = localStorage.getItem(storageKey);
      if (stored === "1") setCollapsed(box, true);
      if (stored === "0") setCollapsed(box, false);
    } catch(e) {}

    hdr.addEventListener("click", function(e){
      e.stopPropagation();
      var willCollapse = !box.classList.contains("sat-about-collapsed");
      setCollapsed(box, willCollapse);
    });

    function doPlace(){ placeBox(mapEl, box); }
    var placeTries = 0, iv = setInterval(function(){ doPlace(); if (++placeTries > 25) clearInterval(iv); }, 120);
    window.addEventListener("resize", doPlace);
    requestAnimationFrame(doPlace);
  }

  if (document.readyState === "loading") { document.addEventListener("DOMContentLoaded", function(){ init(0); }); }
  else { init(0); }
})();
</script>
""")

    html_snippet = tpl.substitute(
        box_id=box_id,
        header_id=header_id,
        collapsed_class=collapsed_class,
        issue_url=issue_url,
        issue_number=str(issue_number),
        map_name=_html.escape(map_name),
        created_date=_html.escape(created_date),
        links_html=links_html,
        map_dom_id=map_dom_id,
    )
    m.get_root().html.add_child(folium.Element(html_snippet))

# =========================
# 1) Fetch ferry stops
# =========================
if FETCH_FROM_WD:
    print("Hämtar färjelägen från Wikidata…")
    print(SPARQL)
    df_ferry = fetch_wd_to_csv(SPARQL, "ferry_stops_wd.csv")
else:
    if not os.path.exists("ferry_stops_wd.csv"):
        raise FileNotFoundError("Hittar inte ferry_stops_wd.csv – kör FETCH_FROM_WD=True eller lägg filen bredvid scriptet.")
    df_ferry = pd.read_csv("ferry_stops_wd.csv")

print(f"Färjelägen: {len(df_ferry)}")

# =========================
# 2) Read SAT_full.geojson
# =========================
if not os.path.exists(SAT_GEOJSON_PATH):
    raise FileNotFoundError(f"Saknar {SAT_GEOJSON_PATH}.")
with open(SAT_GEOJSON_PATH, "r", encoding="utf-8") as f:
    sat_geo = json.load(f)

# =========================
# 3) Init map
# =========================
minlat, minlon, maxlat, maxlon = bounds_from_geojson(sat_geo)
center = [(minlat+maxlat)/2, (minlon+maxlon)/2]

m = folium.Map(location=center, zoom_start=10, tiles="CartoDB positron", control_scale=True)
Fullscreen().add_to(m)
MiniMap(toggle_display=True, position="bottomleft").add_to(m)
MeasureControl(position='topleft', primary_length_unit='kilometers', secondary_length_unit='meters').add_to(m)
MousePosition(position='bottomright', separator=' | ', num_digits=5, prefix='Lat/Lon:').add_to(m)

# SAT trail overlay
sat_fg = folium.FeatureGroup(name="SAT spår (SAT_full.geojson)", show=True)
folium.GeoJson(
    sat_geo,
    name="SAT_full",
    style_function=lambda feat: {"color":"#e4572e","weight":5,"opacity":0.9},
).add_to(sat_fg)
sat_fg.add_to(m)

# =========================
# 4) Add ferry stops
# =========================
ferry_fg = folium.FeatureGroup(name="Färjelägen", show=True)

for _, row in df_ferry.iterrows():
    lat, lon = row.get("lat"), row.get("lon")
    if pd.isna(lat) or pd.isna(lon):
        continue

    itemLabel   = safe(row.get("itemLabel")) or "Färjeläge"
    etappLabel  = safe(row.get("etappLabel"))
    img         = safe(row.get("img"))
    osmnode     = safe(row.get("OSMnode"))
    osmway      = safe(row.get("OSMway"))
    osmrel      = safe(row.get("OSMrel"))
    googlemap   = safe(row.get("googlemap"))
    wd_url      = safe(row.get("item"))

    kartIcon    = (row.get("kartIcon") or "").lower()
    fa_icon     = KARTO_MAP.get(kartIcon, "info-sign")
    hexcolor    = safe(row.get("hex"))
    if hexcolor and not hexcolor.startswith("#"):
        hexcolor = "#" + hexcolor

    # Links
    links = []
    links.append(f"▶️ <a href='https://youtu.be/k55c4ru9y5c' target='_blank'>Video about the ferry stop map</a>")
    if googlemap: links.append(f"📍 <a href='{html.escape(googlemap)}' target='_blank'>Google Maps</a>")
    if osmnode:   links.append(f"🗺️ <a href='https://www.openstreetmap.org/node/{osmnode}'>OSM node</a>")
    if osmway:    links.append(f"🗺️ <a href='https://www.openstreetmap.org/way/{osmway}'>OSM way</a>")
    if osmrel:    links.append(f"🗺️ <a href='https://www.openstreetmap.org/relation/{osmrel}'>OSM rel</a>")
    if wd_url:    links.append(f"🧠 <a href='{html.escape(wd_url)}' target='_blank'>Wikidata</a>")

    links_html = "<ul style='list-style:none; padding:0; margin:6px 0 0 0; font-size:13px; line-height:1.4;'>"
    for l in links:
        links_html += f"<li style='margin:2px 0;'>{l}</li>"
    links_html += "</ul>"

    # Image
    img_html = ""
    if img:
        img_html = f"""
        <div style="margin:8px 0;">
          <img src="{html.escape(img)}" style="max-width:280px; border-radius:8px; box-shadow:0 4px 12px rgba(0,0,0,.25);" referrerpolicy="no-referrer">
        </div>
        """

    html_popup = f"""
    <div style="font-family:system-ui; max-width:330px; line-height:1.35;">
      <div style="font-weight:600; font-size:15px; margin-bottom:4px;">{html.escape(itemLabel)}</div>
      <div style="font-size:12px; opacity:0.7; margin-bottom:6px;">🚢 Ferry stop (SAT)</div>
      {img_html}
      <div style="font-size:13px; margin:2px 0;">🧭 <b>Delsträcka:</b> {html.escape(etappLabel) if etappLabel else ""}</div>
      {links_html}
    </div>
    """

    # Main marker
    folium.Marker(
        location=[float(lat), float(lon)],
        icon=folium.Icon(icon=fa_icon, color="blue"),
        popup=folium.Popup(html_popup, max_width=380)
    ).add_to(ferry_fg)

    # Overlay with Wikidata-defined color
    if hexcolor:
        folium.CircleMarker(
            location=[float(lat), float(lon)],
            radius=10,
            color=hexcolor,
            fill=True,
            fill_color=hexcolor,
            fill_opacity=0.9
        ).add_to(ferry_fg)

ferry_fg.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

add_about_box(m, issue_number=187, map_name="SAT Ferry stops")


# =========================
# 5) Save
# =========================
os.makedirs("output", exist_ok=True)

ts = datetime.now().strftime("%Y%m%d_%H%M%S")
out_ts = os.path.join("output", f"{OUTPUT_PREFIX}_{ts}.html")
out_latest = os.path.join("output", f"{OUTPUT_PREFIX}_latest.html")

m.save(out_ts)
m.save(out_latest)

print("Klar:")
print("  ", out_ts)
print("  ", out_latest)


Hämtar färjelägen från Wikidata…

SELECT ?item ?itemLabel ?coord ?img ?OSMnode ?OSMway ?OSMrel 
       ?etapp ?etappLabel ?googlemap ?type ?typeLabel ?kartIcon ?hex
WHERE {
  ?etapp wdt:P361 wd:Q131318799.
  ?etapp wdt:P1427 ?item.
  ?item wdt:P31 ?type.

  OPTIONAL { ?item wdt:P625 ?coord. }
  OPTIONAL { ?item wdt:P18 ?img. }
  OPTIONAL { ?item wdt:P11693 ?OSMnode }
  OPTIONAL { ?item wdt:P10689 ?OSMway }
  OPTIONAL { ?item wdt:P402   ?OSMrel }
  OPTIONAL { ?item wdt:P3749 ?gid. 
             BIND(URI(CONCAT("https://www.google.com/maps?cid=",?gid)) AS ?googlemap) }

  OPTIONAL { ?type wdt:P1343 ?kartIcon. }
  OPTIONAL { ?type wdt:P465 ?hex. }

  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?etapp ?itemLabel

Färjelägen: 43
Klar:
   output/187_WD_OSM_ferry_stops_20251105_054311.html
   output/187_WD_OSM_ferry_stops_latest.html


In [8]:
links

["▶️ <a href='https://youtu.be/k55c4ru9y5c' target='_blank'>Video about the ferry stop map</a>",
 "📍 <a href='https://www.google.com/maps?cid=12670935218649844604' target='_blank'>Google Maps</a>",
 "🗺️ <a href='https://www.openstreetmap.org/node/476790748'>OSM node</a>",
 "🧠 <a href='http://www.wikidata.org/entity/Q134370050' target='_blank'>Wikidata</a>"]

In [9]:
end_time = time.time()
duration = end_time - start_time
print(f"Finished in {duration:.2f} seconds.")


Finished in 2.21 seconds.
